In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm as log_progress
import os
from IPython.display import JSON
import json
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

import helpers
import api 

In [2]:
df = pd.read_csv('all_provas_enem_areas.csv')

In [3]:
df.loc[~df['description'].isna()].head()

,id,year,context,question,options,correct_answer,CE,DS,EK,IC,MR,TC,image,IU,ledor,figures,description,area,cor
1404,1,2023,"No man is an island,\nEntire of itself;\nEvery...","Nesse poema, a expressão “No man is an island”...","{'A': 'medo da morte.', 'B': 'ideia de conexão...",{'B': 'ideia de conexão.'},NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[],"LINGUAGENS, CÓDIGOS E SUAS TECNOLOGIAS",LARANJA
1405,2,2023,## Things We Carry on the Sea\nWe carry tears ...,"Ao retratar a trajetória de refugiados, o poem...","{'A': 'risco de choques culturais.', 'B': 'imp...",{'D': 'existência de experiências plurais.'},NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[],"LINGUAGENS, CÓDIGOS E SUAS TECNOLOGIAS",LARANJA
1406,3,2023,[[placeholder]],Esse cartaz de campanha sugere que,"{'A': 'os lixões precisam de ampliação.', 'B':...",{'B': 'o desperdício degrada o ambiente.'},NaN,NaN,NaN,NaN,NaN,NaN,Yes,True,True,['https://raw.githubusercontent.com/piresramon...,['Descrição do cartaz: Foto de parte de um lix...,"LINGUAGENS, CÓDIGOS E SUAS TECNOLOGIAS",LARANJA
1407,4,2023,[[placeholder]],Ao retratar o ambiente de trabalho em um escri...,"{'A': 'criticar um padrão de vestimenta.', 'B'...",{'B': 'destacar a falta de diversidade.'},NaN,NaN,NaN,NaN,NaN,NaN,Yes,True,True,['https://raw.githubusercontent.com/piresramon...,"['Descrição do cartum: Em um escritório, dois ...","LINGUAGENS, CÓDIGOS E SUAS TECNOLOGIAS",LARANJA
1408,5,2023,(fragmento)\nSpanglish\nTato Laviera\npues est...,"Nesse poema de Tato Laviera, o eu lírico desta...","{'A': 'convergência linguístico-cultural.', 'B...",{'A': 'convergência linguístico-cultural.'},NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[],"LINGUAGENS, CÓDIGOS E SUAS TECNOLOGIAS",LARANJA


In [4]:
df['system_answer'] = ""
df['context'] = df.apply(lambda row: row['context'] + ' ' + row['description'] if pd.notna(row['description']) else row['context'], axis=1)

In [5]:
df.head()

,id,year,context,question,options,correct_answer,CE,DS,EK,IC,MR,TC,image,IU,ledor,figures,description,area,cor,system_answer
0,1,2013,\nDistribuição espacial atual da população bra...,\nOs mapas representam distintos padrões ...,"{'A': ' fertilização natural dos solos.\n', '...",{'E': ' concentração histórica da urbanização...,No,No,No,Yes,No,No,Yes,NaN,NaN,NaN,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,
1,2,2013,\nTrata-se de um gigantesco movimento de const...,\nA dinâmica de transformação das cidades...,{'A': ' crescimento da população urbana e aum...,{'A': ' crescimento da população urbana e aum...,No,No,Yes,No,No,No,No,NaN,NaN,NaN,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,
2,3,2013,"\nNo dia 1º de julho de 2012, a cidade do Rio ...",\nO reconhecimento da paisagem em questão...,{'A': ' presença do corpo artístico local.\n'...,{'E': ' relação sociedade-natureza de caráter...,No,No,Yes,No,No,Yes,No,NaN,NaN,NaN,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,
3,4,2013,TEXTO I\n\nHá já algum tempo eu me aperc...,\nA exposição e a análise do projeto cartesian...,{'A': ' retomar o método da tradição para edi...,{'B': ' questionar de forma ampla e prof...,No,No,No,No,No,Yes,No,NaN,NaN,NaN,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,
4,5,2013,"\nDe ponta a ponta, é tudo praia-palma, ...",\nA carta de Pero Vaz de Caminha permit...,{'A': ' Valorizar a catequese a ser real...,{'A': ' Valorizar a catequese a ser real...,No,No,No,No,No,Yes,No,NaN,NaN,NaN,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,


In [6]:
df_MCQ = pd.DataFrame(columns=['area', 'year', 'context', 'question', 'options', 'correct_answer', 'bloom_level', 'difficult_level', 'relevance', 'adherence', 'grammar', 'answerability', 'correctness', 'feedback', 'rouge', 'bleu', 'system_answer', 'model'])

In [7]:
df_MCQ

,area,year,context,question,options,correct_answer,bloom_level,difficult_level,relevance,adherence,grammar,answerability,correctness,feedback,rouge,bleu,system_answer,model


In [8]:
for model in ["gpt-3.5-turbo"]:#, "gpt-3.5", "gpt-4-32k", "gpt-4"]:
    c=0
    gpt = api.GPT(model=model)
    for index, row in log_progress(df.loc[(df['year']==2013) & (df['image'] == 'No')].iterrows()):
        try:
            c+=1
            print(c)
            system_context, user_prompt = helpers.prompt_builder_PT([[row['area'], row['context']]])
            messages=[{"role":"system","content": system_context},
                  {"role":"user","content": user_prompt}]
            res = gpt.chat(messages)
            sleep(0.01)
            response = json.loads(res)
    
            for question in response['questoes'][0]['perguntas']:
                evals = helpers.evaluate_generated_mcq(response['questoes'][0]['context'], question['pergunta'])
                dic = {'area': row['area'], 'year': row['year'], 'context':row['context']}
                dic['question'] = question['pergunta']
                options = [{k: v for k, v in dict(option).items() if k != 'correta'} for option in question['options']]
                options =  dict(item for d in options for item in d.items())
                dic['options'] = str(options)
                dic['correct_answer'] = question['correct_answer'][0]
                dic['bloom_level'] = question['bloom_level']
                dic['difficult_level'] = question['difficult_level']
                eval_pt = question
                eval_pt['area'] = response['questoes'][0]['area']
                eval_pt['context'] = response['questoes'][0]['context']

                system_context, user_prompt = helpers.evaluator_builder_PT_(eval_pt)
                messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
                response_eval = json.loads(gpt.chat(messages))
                sleep(0.01)
                dic['relevance'] = response_eval['relevance']
                dic['adherence'] = response_eval['adherence']
                dic['grammar'] = response_eval['grammar']
                dic['answerability'] = response_eval['answerability']
                dic['correctness'] = response_eval['correctness']
                dic['feedback'] = response_eval['feedback']
            
                dic['rouge'] = str(evals[0]['rougeL'])
                dic['bleu'] = evals[1]

                system_context, user_prompt = helpers.solve_builder_PT_([response['questoes'][0]['context'], question['pergunta'], str(options)])
                messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
                response_answer = json.loads(gpt.chat(messages))
                sleep(0.01)
                dic['system_answer'] = response_answer
        
                dic['model'] = model
        
                df_MCQ.loc[len(df_MCQ)] = dic
    
            system_context, user_prompt = helpers.solve_builder_PT_([row['context'], row['question'], row['options']])
            messages=[{"role":"system","content": system_context},
                  {"role":"user","content": user_prompt}]
            #response_resposta = json.loads(gpt.chat(messages))
            response_resposta = gpt.chat(messages)
            df.loc[index, 'system_answer'] = df.loc[index, 'system_answer']+'['+ model + ': ' + response_resposta+']'
            sleep(0.01)
        except:
            pass
 

0it [00:00, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [11]:
df_MCQ.to_csv('MCQ_solved_2.csv', index=False)

In [12]:
df.to_csv('all_provas_enem_areas_sys_solved.csv', index=False)